## Notebook used to calculate spectrum of $\Phi$ values using modified PyPhi

#### First get all the concepts for the original TPM

In [9]:
import pyphi
import numpy as np
from pyphi import Direction
from pyphi import utils
from pyphi import cache
from pyphi import models

# Transition probability matrix. Saying where each state (little-endian binary) goes
tpm = np.array([
    [0.,0.,0.],
    [0.,0.,0.],
    [1.,0.,0.],
    [1.,0.,1.],
    [0.,1.,0.],
    [0.,1.,0.],
    [1.,1.,0.],
    [1.,1.,1.]
])

# Set up network object
network = pyphi.Network(tpm, node_labels=['R','C','P'])
print("Network = ",network.node_labels)

# Put the system into a given state
state = (0,0,0)
nodes = ['R','C','P']
subsystem = pyphi.Subsystem(network, state, nodes)


## Find all concepts for a given subsystem
original_concepts = []
mechanisms = utils.powerset(subsystem.node_indices, nonempty=True)
for mechanism in mechanisms:
#     print("MECHANISM = ",mechanism)
    
    # Find the CORE cause(s) for a given mechanism
    direction = Direction.CAUSE
    past_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_causes = [subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews]

    if not past_purviews:
        core_cause = [_null_ria(direction, mechanism, ())]
    else:
        core_cause = max(all_causes)
        if core_cause.phi > 0.:
            all_core_causes = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_cause.phi]
        else:
            all_core_causes = [core_cause]
            
#     print("Core Cause(s) for Mechanism : ",all_core_causes)
    
    # Find the CORE effect(s) for a given mechanism
    direction = Direction.EFFECT
    future_purviews = subsystem.potential_purviews(direction, mechanism, purviews=False)
    all_effects = [subsystem.find_mip(direction, mechanism, purview) for purview in future_purviews]

    if not future_purviews:
        core_effect = [_null_ria(direction, mechanism, ())]
    else:
        core_effect = max(all_effects)
        if core_effect.phi > 0.:
            all_core_effects = [value for index,value in enumerate(subsystem.find_mip(direction, mechanism, purview) for purview in past_purviews) if value.phi == core_effect.phi]
        else:
            all_core_effects = [core_effect]
            
#     print("Core Effects(s) for Mechanism : ",all_core_effects)

    ## Now we need to build concepts for all possible core cause and core effect combinations
    for each_cause in all_core_causes:
        MIC = models.MaximallyIrreducibleCause(each_cause)
        for each_effect in all_core_effects:
            MIE = models.MaximallyIrreducibleEffect(each_effect)
            
            concept = models.Concept(mechanism = mechanism, cause = MIC, effect = MIE, subsystem = subsystem)
            original_concepts.append(concept)
            
            
# class CauseEffectStructure(cmp.Orderable, collections.Sequence):
#     """A collection of concepts."""

#     def __init__(self, concepts=(), subsystem=None, time=None):
#         # Normalize the order of concepts
#         self.concepts = tuple(sorted(concepts, key=_concept_sort_key))
#         self.subsystem = subsystem
#         self.time = time
        
original_CES = models.CauseEffectStructure(concepts=original_concepts)
print(original_CES)
# for each_concept in original_concepts:
#     print(each_concept)


Network =  NodeLabels(('R', 'C', 'P'))
════════════════════════════════════════════════════════════
            Cause-effect structure (9 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
             Concept: Mechanism = [R], φ = 1/4              
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 1/2                 ││  φ = 1/4                 │  
  │  Purview = [C]           ││  Purview = [P]           │  
  │  MIP:                    ││  MIP:                    │  
  │     ∅     R              ││     ∅     R              │  
  │    ─── ✕ ───             ││    ─── ✕ ───             │  
  │     C     ∅              ││     P     ∅              │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌──────────

#### Now cut the original TPM and find the new set of concepts. Get the Phi value and repeat the cut

In [7]:
concept.expand_cause_repertoire((0,1,2,))

array([[[0.5, 0. ],
        [0. , 0. ]],

       [[0.5, 0. ],
        [0. , 0. ]]])